# Getting the dataset
A script has been provided to download all the datasets required for running the below examples.
It will dowload and unzip the WikiQA Corpus and the Quora Duplicate Questions dataset.

In [ ]:
!python experimental_data/get_data.py

## Installing dependencies for running the Similarity Learning task

In [1]:
import os
import csv
import re
from gensim.models.experimental.drmm_tks import DRMM_TKS
from gensim.utils import simple_preprocess

Using TensorFlow backend.
2018-07-08 18:09:54,206 : INFO : 'pattern' package not found; tag filters are not available for English


## Data Format

We have to provide data in a format which is understood by the model.
The model understands sentences as a list of words. 
Further, we need to give a :
 1. Queries List
 2. Candidate Document List
 3. Correct Label List

1 is a list of list of words
2 and 3 is actually a list of list of list of words/ints

Example:
```
queries = ["When was Abraham Lincoln born ?".split(), 
            "When was the first World War ?".split()]
docs = [
		 ["Abraham Lincoln was the president of the United States of America".split(),
		 "He was born in 1809".split()],
		 ["The first world war was bad".split(),
		 "It was fought in 1914".split(),
		 "There were over a million deaths".split()]
       ]
labels = [[0,
           1],
		  [0,
           1,
           0]
          ]
```

## About the dataset : WikiQA

The WikiQA corpus is a set of question-answer pairs in which for every query there are several candidate documents of which none, one or more documents might be relevant.
Relevance is purely binary, i.e., 1: relavant, 0: not relevant

Sample data:

QuestionID | Question | DocumentID | DocumentTitle | SentenceID | Sentence | Label
-- | -- | -- | -- | -- | -- | --
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-0 | A partly submerged glacier cave on Perito Moreno Glacier . | 0
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-1 | The ice facade is approximately 60 m high | 0
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-2 | Ice formations in the Titlis glacier cave | 0
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-3 | A glacier cave is a cave formed within the ice of a glacier . | 1
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-4 | Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-0 | In physics , circular motion is a movement of an object along the circumference of a circle or rotation along a circular path. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-1 | It can be uniform, with constant angular rate of rotation (and constant speed), or non-uniform with a changing rate of rotation. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-2 | The rotation around a fixed axis of a three-dimensional body involves circular motion of its parts. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-3 | The equations of motion describe the movement of the center of mass of a body. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-4 | Examples of circular motion include: an artificial satellite orbiting the Earth at constant height, a stone which is tied to a rope and is being swung in circles, a car turning through a curve in a race track , an electron moving perpendicular to a uniform magnetic field , and a gear turning inside a mechanism. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-5 | Since the object's velocity vector is constantly changing direction, the moving object is undergoing acceleration by a centripetal force in the direction of the center of rotation. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-6 | Without this acceleration, the object would move in a straight line, according to Newton's laws of motion . | 0

## Data Preprocessing
We need to take the above text and make it into `queries, docs, labels` form. For this, we will create an iterable object with the below class which will allow the data to be streamed into the model as the need arises.

In [2]:
class MyWikiIterable:
    """"Yields the next data point in the data set based on the `iter_type`
    
    Based on `iter_type` the object can yield the following:
        'query' : list of str words
        'doc' : list of docs
                    where a doc is a list of str words
        'label' : list of int
                  The relevance between adjacent queries and docs
    """

    def __init__(self, iter_type, fpath):
        """
        Parameters
        ----------
        iter_type : {'query', 'doc', 'label'}
            the type of iterable to be yielded
        fpath : str
            path to the dataset
        """

        # To map the `iter_type` to an index
        self.type_translator = {'query': 0, 'doc': 1, 'label': 2}
        self.iter_type = iter_type

        with open(fpath, encoding='utf8') as tsv_file:
            tsv_reader = csv.reader(tsv_file, delimiter='\t', quoting=csv.QUOTE_NONE)
            self.data_rows = []
            self.data_rows = [row for row in tsv_reader]

    def preprocess_sent(self, sent):
        """Utility function to lower, strip and tokenize each sentence
        Replace this function if you want to handle preprocessing differently"""

        return simple_preprocess(sent)

    def __iter__(self):
        # Defining some consants for .tsv reading
        # They represent the columns of the respective values
        QUESTION_ID_INDEX = 0
        QUESTION_INDEX = 1
        ANSWER_INDEX = 5
        LABEL_INDEX = 6


        # The group of documents and labels that belong to one question
        document_group = []
        label_group = []

        # Number of relevant documents per query
        n_relevant_docs = 0
        # Number of filtered docs (query-doc pairs which have zero relevant docs)
        n_filtered_docs = 0

        # The data
        queries = []
        docs = []
        labels = []

        # The code below goes through the data line by line
        # It checks the current document id with the next document id
        for i, line in enumerate(self.data_rows[1:], start=1):

            if i < len(self.data_rows) - 1:  # check if out of bounds might occur

                if self.data_rows[i][QUESTION_ID_INDEX] == self.data_rows[i + 1][QUESTION_ID_INDEX]:
                    document_group.append(self.preprocess_sent(self.data_rows[i][ANSWER_INDEX]))
                    label_group.append(int(self.data_rows[i][LABEL_INDEX]))
                    n_relevant_docs += int(self.data_rows[i][LABEL_INDEX])
                else:
                    document_group.append(self.preprocess_sent(self.data_rows[i][ANSWER_INDEX]))
                    label_group.append(int(self.data_rows[i][LABEL_INDEX]))
                    n_relevant_docs += int(self.data_rows[i][LABEL_INDEX])

                    if n_relevant_docs > 0:
                        docs.append(document_group)
                        labels.append(label_group)
                        queries.append(self.preprocess_sent(self.data_rows[i][QUESTION_INDEX]))

                        yield [queries[-1], document_group, label_group][self.type_translator[self.iter_type]]
                    else:
                        n_filtered_docs += 1

                    n_relevant_docs = 0
                    document_group = []
                    label_group = []

            else:
                # If we are on the last line
                document_group.append(self.preprocess_sent(self.data_rows[i][ANSWER_INDEX]))
                label_group.append(int(self.data_rows[i][LABEL_INDEX]))
                n_relevant_docs += int(self.data_rows[i][LABEL_INDEX])

                if n_relevant_docs > 0:
                    docs.append(document_group)
                    labels.append(label_group)
                    queries.append(self.preprocess_sent(self.data_rows[i][QUESTION_INDEX]))
                    yield [queries[-1], document_group, label_group][self.type_translator[self.iter_type]]
                else:
                    n_filtered_docs += 1
                    n_relevant_docs = 0
        print("Skipped ", n_filtered_docs)

In [3]:
l_iterable = MyWikiIterable('label', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-train.tsv'))
for l in l_iterable:
    print(l)
    if sum(l) == 0:
        bdfbv

[0, 0, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0]
[1, 1]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 0]
[0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[1, 0]
[

[1, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1]
[1, 0, 0, 0, 0]
[1, 1]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0

Now, will use the class to create objects of the training iterable

In [4]:
q_iterable = MyWikiIterable('query', os.path.join( 'experimental_data', 'WikiQACorpus', 'WikiQA-train.tsv'))
d_iterable = MyWikiIterable('doc', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-train.tsv'))
l_iterable = MyWikiIterable('label', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-train.tsv'))

We will also initialize some validation iterables
Note: the path has `dev` in it

In [5]:
q_val_iterable = MyWikiIterable('query', os.path.join( 'experimental_data', 'WikiQACorpus', 'WikiQA-dev.tsv'))
d_val_iterable = MyWikiIterable('doc', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-dev.tsv'))
l_val_iterable = MyWikiIterable('label', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-dev.tsv'))

# Using word embeddings
We also need to get the word embeddings for the training. For this, we will use the Glove Embeddings.
Luckily, [gensim-data](https://github.com/RaRe-Technologies/gensim-data) provides an easy interface for it.

We will use the [KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) object that we for from gensim-data api and pass it as the `word_embedding` parameter in the model.

In [6]:
import gensim.downloader as api
kv_model = api.load("glove-wiki-gigaword-300")

2018-07-08 18:10:35,382 : INFO : loading projection weights from /home/aneeshj/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
2018-07-08 18:12:21,752 : INFO : loaded (400000, 300) matrix from /home/aneeshj/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz


# Training the Model
Now that we have the preprocessed extracted data and word embeddings, training the model just takes one line:

In [14]:
# Train the model
drmm_tks_model = DRMM_TKS(q_iterable, d_iterable, l_iterable, word_embedding=kv_model,
                                epochs=3, validation_data=[q_val_iterable, d_val_iterable, l_val_iterable])

2018-07-08 18:17:46,569 : INFO : Starting Vocab Build


Skipped  1245


2018-07-08 18:17:48,161 : INFO : Vocab Build Complete
2018-07-08 18:17:48,161 : INFO : Vocab Size is 18814
2018-07-08 18:17:48,164 : INFO : Building embedding index using KeyedVector pretrained word embeddings
2018-07-08 18:17:48,165 : INFO : The embeddings_index built from the given file has 400000 words of 300 dimensions
2018-07-08 18:17:48,166 : INFO : Building the Embedding Matrix for the model's Embedding Layer


Skipped  1245


2018-07-08 18:17:48,360 : INFO : There are 642 words out of 18814 (3.41%) not in the embeddings. Setting them to random
2018-07-08 18:17:48,360 : INFO : Adding additional words from the embedding file to embedding matrix
2018-07-08 18:17:53,194 : INFO : Normalizing the word embeddings
2018-07-08 18:18:54,026 : INFO : Embedding Matrix build complete. It now has shape (400644, 300)
2018-07-08 18:19:03,500 : INFO : Pad word has been set to index 400642
2018-07-08 18:19:04,378 : INFO : Unknown word has been set to index 400643
2018-07-08 18:19:04,682 : INFO : Embedding index build complete
2018-07-08 18:19:33,261 : INFO : Input is an iterable amd will be streamed
2018-07-08 18:21:56,344 : INFO : __________________________________________________________________________________________________
2018-07-08 18:22:04,752 : INFO : Layer (type)                    Output Shape         Param #     Connected to                     
2018-07-08 18:22:04,861 : INFO : ===================================

...

2018-07-08 18:26:19,741 : INFO : MAP: 0.60
2018-07-08 18:26:19,747 : INFO : nDCG@1 : 0.44
2018-07-08 18:26:19,754 : INFO : nDCG@3 : 0.61
2018-07-08 18:26:19,766 : INFO : nDCG@5 : 0.65
2018-07-08 18:26:19,775 : INFO : nDCG@10 : 0.69
2018-07-08 18:26:19,780 : INFO : nDCG@20 : 0.70


Epoch 2/3
888/900 [============================>.] - ETA: 1s - loss: 0.8238 - acc: 0.3736Skipped  1245
SAMPLA RE!!!!!!!!!!!!!!!!!! 8995
900/900 [==============================] - 85s 94ms/step - loss: 0.8251 - acc: 0.3748


2018-07-08 18:27:49,010 : INFO : MAP: 0.63
2018-07-08 18:27:49,016 : INFO : nDCG@1 : 0.48
2018-07-08 18:27:49,021 : INFO : nDCG@3 : 0.63
2018-07-08 18:27:49,028 : INFO : nDCG@5 : 0.68
2018-07-08 18:27:49,034 : INFO : nDCG@10 : 0.72
2018-07-08 18:27:49,041 : INFO : nDCG@20 : 0.72


Epoch 3/3
888/900 [============================>.] - ETA: 1s - loss: 0.8002 - acc: 0.4135Skipped  1245
SAMPLA RE!!!!!!!!!!!!!!!!!! 8995
900/900 [==============================] - 86s 96ms/step - loss: 0.8004 - acc: 0.4150


2018-07-08 18:29:20,071 : INFO : MAP: 0.62
2018-07-08 18:29:20,077 : INFO : nDCG@1 : 0.46
2018-07-08 18:29:20,082 : INFO : nDCG@3 : 0.63
2018-07-08 18:29:20,090 : INFO : nDCG@5 : 0.67
2018-07-08 18:29:20,096 : INFO : nDCG@10 : 0.71
2018-07-08 18:29:20,102 : INFO : nDCG@20 : 0.72


## Saving and loading the model
The trained model can be saved and loaded from memory for future use.

In [ ]:
drmm_tks_model.save('drmm_tks_model')

In [ ]:
del drmm_tks_model
drmm_tks_model = DRMM_TKS.load('drmm_tks_model')

## Testing the model on new data

The testing of the data can be done on completely unseen data using `model.predict(queries, docs)` where
queries: list of list of words
docs: list of list of list of words

In [31]:
queries = [simple_preprocess("how are glacier caves formed"),
           simple_preprocess("What is AWS")]

docs = [[simple_preprocess("A partly submerged glacier cave on Perito Moreno Glacier"),
        simple_preprocess("A glacier cave is a cave formed within the ice of a glacier")],
       [simple_preprocess("AWS stands for Amazon Web Services"),
        simple_preprocess("AWS was established in 2001"),
        simple_preprocess("It is a cloud service")]]

The predict function returns the similarity between a query-document pair in a list format

For example
```
queries = [q1, q2]
docs = [[d1_1, d1_2],
        [d2_1, d2_2, d2_3]]

model.predict(queries, docs)

Output
------
q1-d1_1 similarity
q1-d1_2 similarity
q2-d2_1 similarity
q2-d2_2 similarity
q2-d2_3 similarity
```

In [32]:
drmm_tks_model.predict(queries, docs)

2018-07-06 19:44:53,879 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-06 19:44:53,880 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-06 19:44:53,906 : INFO : Predictions in the format query, doc, similarity
2018-07-06 19:44:53,957 : INFO : ['how', 'are', 'glacier', 'caves', 'formed']	['partly', 'submerged', 'glacier', 'cave', 'on', 'perito', 'moreno', 'glacier']	0.60059035
2018-07-06 19:44:53,958 : INFO : ['how', 'are', 'glacier', 'caves', 'formed']	['glacier', 'cave', 'is', 'cave', 'formed', 'within', 'the', 'ice', 'of', 'glacier']	0.9258956
2018-07-06 19:44:53,959 : INFO : ['what', 'is', 'aws']	['aws', 'stands', 'for', 'amazon', 'web', 'services']	0.38946432
2018-07-06 19:44:53,959 : INFO : ['what', 'is', 'aws']	['aws', 'was', 'established', 'in']	0.37697053
2018-07-06 19:44:53,960 : INFO : ['what', 'is', 'aws']	['it', 'is', 'cloud', 'service']	0.5015697


array([[0.60059035],
       [0.9258956 ],
       [0.38946432],
       [0.37697053],
       [0.5015697 ]], dtype=float32)

As can be seen from the logs and results above, within each query-document group, the correct answer has the highest score

For example,
In the first group
```
['how', 'are', 'glacier', 'caves', 'formed'] ['partly', 'submerged', 'glacier', 'cave', 'on', 'perito', 'moreno', 'glacier']	0.53
['how', 'are', 'glacier', 'caves', 'formed'] ['glacier', 'cave', 'is', 'cave', 'formed', 'within', 'the', 'ice', 'of', 'glacier']	0.61
```

The correct answer, "glacier cave is cave ..." has the higher score as compared to the first answer
The second pair almost gets it right (WIP)

### Testing on a test set
We can pass a whole dataset and get evaluations based on that. Let's try with the test set of WikiQA Corpus

In [12]:
q_test_iterable = MyWikiIterable('query', os.path.join( 'experimental_data', 'WikiQACorpus', 'WikiQA-test.tsv'))
d_test_iterable = MyWikiIterable('doc', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-test.tsv'))
l_test_iterable = MyWikiIterable('label', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-test.tsv'))

In [34]:
for q, doc, lab in zip(q_test_iterable, d_test_iterable, l_test_iterable):
    for d, l in zip(doc, lab):
        if len(q) ==0 or len(d)==0 :
            print(q,d,l)

Skipped  390


In [18]:
drmm_tks_model.evaluate(q_test_iterable, d_test_iterable, l_test_iterable)

2018-07-08 18:31:45,594 : INFO : Found 21 unknown words. Set them to unknown word index : 400643
2018-07-08 18:31:45,663 : INFO : Found 249 unknown words. Set them to unknown word index : 400643


Skipped  390


2018-07-08 18:31:55,140 : INFO : MAP: 0.57
2018-07-08 18:31:55,155 : INFO : nDCG@1 : 0.43
2018-07-08 18:31:55,174 : INFO : nDCG@3 : 0.58
2018-07-08 18:31:55,189 : INFO : nDCG@5 : 0.65
2018-07-08 18:31:55,201 : INFO : nDCG@10 : 0.68
2018-07-08 18:31:55,211 : INFO : nDCG@20 : 0.70


## Comparing DRMM TKS with other models

It would be good to get an idea of how our model works against some unsupervised models like word2vec and FastText.
For this, we will, given a query-document pair, we will get a vector for the query and document. We can get the similarity between them using the cosine similarity between their vectors.

### For word2vec


In [7]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

In [8]:
def sent2vec(sent, conv_dict):
    vec = []
    for word in sent:
        if word in conv_dict:
            vec.append(conv_dict[word])
    vec = np.array(vec)
    if np.sum(vec) == 0:
        return np.random.random((300,1))
    vec = np.mean(vec, axis=0)
    return vec

In [9]:
import numpy as np
sent2vec(simple_preprocess("aneesjh"), kv_model.wv)

/home/aneeshj/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


array([[2.69996447e-01],
       [1.74878370e-01],
       [2.01714921e-01],
       [8.93567478e-01],
       [8.57299378e-01],
       [9.81997667e-01],
       [3.42151236e-01],
       [1.31718018e-01],
       [1.36217828e-01],
       [7.43161605e-01],
       [1.48885837e-01],
       [8.16323523e-01],
       [5.08442320e-01],
       [8.69770641e-01],
       [7.23417196e-01],
       [5.90799997e-01],
       [2.92285244e-01],
       [5.44479818e-02],
       [1.11422171e-01],
       [3.37522087e-01],
       [4.21111699e-02],
       [5.73770725e-02],
       [8.78533728e-01],
       [6.05013695e-01],
       [1.15063918e-01],
       [3.47707662e-01],
       [1.37459857e-01],
       [2.69619389e-01],
       [6.50323283e-01],
       [6.16153522e-02],
       [1.09051013e-01],
       [5.14038244e-01],
       [6.56012472e-01],
       [4.02121363e-02],
       [8.43769234e-01],
       [9.29815386e-03],
       [9.39422174e-02],
       [6.37361711e-01],
       [4.78272891e-02],
       [7.80214282e-01],


In [28]:
import numpy as np
from gensim.models.experimental import mapk, mean_ndcg

def eval_model(queries, docs, labels, model, _translate_user_data):

    long_label_list = []
    long_pred_list = []
    doc_lens = []
    
    for query, doc, label in zip(queries, docs, labels):
        for d, l in zip(doc, label):
            if 1==2:
                trans_q = _translate_user_data(query, model.wv)
                trans_d = _translate_user_data(d, model.wv)
                long_pred_list.append(cosine_similarity(trans_q, trans_d))
            else:
#                 print(query)
                
                long_pred_list.append(drmm_tks_model.predict(queries= [query], docs=[[d]]))
            long_label_list.append(l)
        doc_lens.append(len(doc))

#     check_doc_len = []
#     for d in docs:
#         check_doc_len.append(len(d))
        
    print(len(long_pred_list))
    print(len(long_label_list))
#     print(long_label_list)
    print(sum(doc_lens))
# #     print(doc_lens)
# #     print(check_doc_len)
    
# #     for i, q, d in zip(range(len(check_doc_len)), queries, docs):
# #         if doc_lens[i] != check_doc_len[i]:
# #             print(q, d)
# #             break

    predictions=long_pred_list
    Y_pred = []
    Y_true = []
    offset = 0
    for doc_size in doc_lens:
        Y_pred.append(predictions[offset: offset + doc_size])
        Y_true.append(long_label_list[offset: offset + doc_size])
        offset += doc_size

    print("MAP: %.2f" % mapk(Y_true, Y_pred))
    for k in [1, 3, 5, 10, 20]:
        print("nDCG@%d : %.2f"% (k, mean_ndcg(Y_true, Y_pred, k=k)))

In [29]:
eval_model(q_test_iterable, d_test_iterable, l_test_iterable, kv_model, sent2vec)

2018-07-08 18:35:21,387 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,389 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,404 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:21,405 : INFO : ['how', 'african', 'americans', 'were', 'immigrated', 'to', 'the', 'us']	['african', 'immigration', 'to', 'the', 'united', 'states', 'refers', 'to', 'immigrants', 'to', 'the', 'united', 'states', 'who', 'are', 'or', 'were', 'nationals', 'of', 'africa']	1.9191406
2018-07-08 18:35:21,406 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,407 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,421 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:21,422 : INFO : ['how', 'african', 'americans', 'were', 'immigrated', 'to', 'the', 'us']	['the', 'term', 'african', 'in', 'the', 'scope', '

2018-07-08 18:35:21,627 : INFO : ['how', 'old', 'was', 'sue', 'lyon', 'when', 'she', 'made', 'lolita']	['due', 'to', 'the', 'mpaa', 'restrictions', 'at', 'the', 'time', 'the', 'film', 'toned', 'down', 'the', 'more', 'provocative', 'aspects', 'of', 'the', 'novel', 'sometimes', 'leaving', 'much', 'to', 'the', 'audience', 'imagination']	0.44260493
2018-07-08 18:35:21,628 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,629 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,649 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:21,650 : INFO : ['how', 'old', 'was', 'sue', 'lyon', 'when', 'she', 'made', 'lolita']	['the', 'actress', 'who', 'played', 'lolita', 'sue', 'lyon', 'was', 'fourteen', 'at', 'the', 'time', 'of', 'filming']	1.3163476
2018-07-08 18:35:21,652 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,655 : INFO : Found 0 unknown wor

2018-07-08 18:35:21,860 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,870 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:21,870 : INFO : ['how', 'are', 'antibodies', 'used', 'in']	['antibodies', 'are', 'glycoproteins', 'belonging', 'to', 'the', 'immunoglobulin', 'superfamily', 'the', 'terms', 'antibody', 'and', 'immunoglobulin', 'are', 'often', 'used', 'interchangeably']	1.0635862
2018-07-08 18:35:21,871 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,872 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:21,885 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:21,885 : INFO : ['how', 'are', 'antibodies', 'used', 'in']	['antibodies', 'are', 'typically', 'made', 'of', 'basic', 'structural', 'units', 'each', 'with', 'two', 'large', 'heavy', 'chains', 'and', 'two', 'small', 'light', 'chains']	0.7801565
2018-07-

2018-07-08 18:35:22,069 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:22,070 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:22,083 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:22,084 : INFO : ['how', 'much', 'is', 'centavos', 'in', 'mexico']	['the', 'peso', 'is', 'subdivided', 'into', 'centavos', 'represented', 'by']	0.66725993
2018-07-08 18:35:22,085 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:22,085 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:22,099 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:22,101 : INFO : ['how', 'much', 'is', 'centavos', 'in', 'mexico']	['the', 'name', 'was', 'originally', 'used', 'in', 'reference', 'to', 'pesos', 'oro', 'gold', 'weights', 'or', 'pesos', 'plata', 'silver', 'weights']	0.5313859
2018-07-08 18:35:22,102 : INFO : Found

2018-07-08 18:35:22,290 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:22,291 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:22,301 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:22,302 : INFO : ['how', 'does', 'interlibrary', 'loan', 'work']	['the', 'user', 'makes', 'request', 'with', 'their', 'local', 'library', 'which', 'acting', 'as', 'an', 'intermediary', 'identifies', 'owners', 'of', 'the', 'desired', 'item', 'places', 'the', 'request', 'receives', 'the', 'item', 'makes', 'it', 'available', 'to', 'the', 'user', 'and', 'arranges', 'for', 'its', 'return']	0.24911867
2018-07-08 18:35:22,302 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:22,303 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:22,313 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:22,314 : INFO : ['ho

...

2018-07-08 18:35:55,551 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,560 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:55,560 : INFO : ['what', 'were', 'important', 'things', 'that', 'douglas', 'macarthur', 'did']	['he', 'led', 'the', 'united', 'nations', 'command', 'in', 'the', 'korean', 'war', 'until', 'he', 'was', 'removed', 'from', 'command', 'by', 'president', 'harry', 'truman', 'on', 'april']	0.49736142
2018-07-08 18:35:55,561 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,562 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,574 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:55,575 : INFO : ['what', 'were', 'important', 'things', 'that', 'douglas', 'macarthur', 'did']	['he', 'later', 'became', 'chairman', 'of', 'the', 'board', 'of', 'remington', 'rand']	0.3120147
2018-07-08 18:35:55,577 : IN

2018-07-08 18:35:55,774 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,781 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:55,781 : INFO : ['what', 'is', 'the', 'latest', 'season', 'of', 'psych']	['psych', 'is', 'an', 'american', 'detective', 'comedy', 'drama', 'television', 'series', 'created', 'by', 'steve', 'franks', 'and', 'broadcast', 'on', 'usa', 'network']	0.8178164
2018-07-08 18:35:55,782 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,783 : INFO : Found 1 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,794 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:55,794 : INFO : ['what', 'is', 'the', 'latest', 'season', 'of', 'psych']	['it', 'is', 'produced', 'by', 'franks', 'and', 'tagline', 'television', 'chris', 'henze', 'kelly', 'kulchak']	0.5445498
2018-07-08 18:35:55,796 : INFO : Found 0 unknown words. Set them to

2018-07-08 18:35:55,970 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,972 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,982 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:55,982 : INFO : ['what', 'is', 'the', 'main', 'component', 'of', 'vaccines']	['vaccines', 'may', 'be', 'prophylactic', 'example', 'to', 'prevent', 'or', 'ameliorate', 'the', 'effects', 'of', 'future', 'infection', 'by', 'any', 'natural', 'or', 'wild', 'pathogen', 'or', 'therapeutic', 'vaccines', 'against', 'cancer', 'are', 'also', 'being', 'investigated', 'see', 'cancer', 'vaccine']	1.338979
2018-07-08 18:35:55,983 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,984 : INFO : Found 1 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:55,992 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:55,993 : INFO : ['what'

2018-07-08 18:35:56,179 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:56,180 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:56,191 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:56,192 : INFO : ['who', 'is', 'mary', 'matalin', 'married', 'to']	['matalin', 'has', 'been', 'chief', 'editor', 'of', 'threshold', 'editions', 'conservative', 'publishing', 'imprint', 'at', 'simon', 'schuster', 'since', 'march']	0.39624417
2018-07-08 18:35:56,193 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:56,194 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-08 18:35:56,205 : INFO : Predictions in the format query, doc, similarity
2018-07-08 18:35:56,207 : INFO : ['who', 'is', 'mary', 'matalin', 'married', 'to']	['she', 'is', 'married', 'to', 'democratic', 'political', 'consultant', 'james', 'carville']	0.8728301
2018-07-0

Skipped  390
2334
2334
2334
MAP: 0.57
nDCG@1 : 0.43
nDCG@3 : 0.58
nDCG@5 : 0.65
nDCG@10 : 0.68
nDCG@20 : 0.70


Let's compare that with our model

In [ ]:
drmm_tks_model.evaluate(q_test_iterable, d_test_iterable, l_test_iterable)

While the accuracy isn't any better, it is worse, this is still a Work In Progress and we hope to improve it further soon.